In [1]:
import dataset as dataset
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.tree import tree

/home/juan/code/manos-poker/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('../data/poker-hand-tiny.csv')
# df = df[df.Mano != 9]
# df = df[df.Mano != 8]
# df = df[df.Mano != 7]
# df = df[df.Mano != 6]
# df = df[df.Mano != 5]
print(df)
print(df.groupby('Mano').size())

        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
0        1   1   1  13   2   4   2   3   1  12     0
1        3  12   3   2   3  11   4   5   2   5     1
2        1   9   4   6   1   4   3   2   3   9     1
3        1   4   3  13   2  13   2   1   3   6     1
4        3  10   2   7   1   2   2  11   4   9     0
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   ...
299995   2  13   4  11   3   4   4   4   2   4     3
299996   3   5   4  12   2   8   4   8   2  13     1
299997   1  10   1   5   1   4   4   9   3  13     0
299998   2   6   3   4   2   3   4   2   2  13     0
299999   4   6   4   1   3   4   1   8   4  11     0

[300000 rows x 11 columns]
Mano
0    150289
1    126833
2     14395
3      6238
4      1156
5       598
6       423
7        62
8         4
9         2
dtype: int64


In [3]:
# % for train % for test
numTrain = int(0.9*len(df.index))
# splitting dataframe by row index
data_train = df.iloc[:numTrain,:]
data_test = df.iloc[numTrain:,:]
print(data_train)
print(data_test)

        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
0        1   1   1  13   2   4   2   3   1  12     0
1        3  12   3   2   3  11   4   5   2   5     1
2        1   9   4   6   1   4   3   2   3   9     1
3        1   4   3  13   2  13   2   1   3   6     1
4        3  10   2   7   1   2   2  11   4   9     0
...     ..  ..  ..  ..  ..  ..  ..  ..  ..  ..   ...
269995   3  10   3   5   1   3   1   9   4   1     0
269996   4   5   4   6   4  13   4   3   1   5     1
269997   3   3   1  12   3   5   2  12   2   3     2
269998   4   3   2   2   4   4   1  10   4   7     0
269999   3   1   3  12   4  13   4   6   2  13     1

[270000 rows x 11 columns]
        P1  V1  P2  V2  P3  V3  P4  V4  P5  V5  Mano
270000   1  11   4  10   4   8   3  13   2   6     0
270001   1   1   2  10   2   7   4   2   4   5     0
270002   2  13   4  12   3   7   2   4   4   3     0
270003   2   6   2   9   4   9   3   1   3   2     1
270004   1   7   4   5   1   3   4   7   4   8     1
...     ..  ..  ..

In [4]:
print(data_train.groupby('Mano').size())
print(data_test.groupby('Mano').size())

Mano
0    135456
1    113989
2     12932
3      5608
4      1041
5       534
6       382
7        54
8         3
9         1
dtype: int64
Mano
0    14833
1    12844
2     1463
3      630
4      115
5       64
6       41
7        8
8        1
9        1
dtype: int64


In [5]:
# # SMOTE
# X = data_train.loc[:, data_train.columns != 'Mano']
# y = data_train.Mano
# X_res, y_res = SMOTE().fit_resample(X, y)
# data_train = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)
# print(data_train)



In [6]:
# Ready the Train Data
array_train = data_train.values
data_train = array_train[:, 0:10]
label_train = array_train[:, 10]
print(array_train)
print(data_train)
print(label_train)

# Ready the Test Data
array_test = data_test.values
data_test = array_test[:, 0:10]
label_test = array_test[:, 10]
print(array_test)
print(data_test)
print(label_test)

[[ 1  1  1 ...  1 12  0]
 [ 3 12  3 ...  2  5  1]
 [ 1  9  4 ...  3  9  1]
 ...
 [ 3  3  1 ...  2  3  2]
 [ 4  3  2 ...  4  7  0]
 [ 3  1  3 ...  2 13  1]]
[[ 1  1  1 ...  3  1 12]
 [ 3 12  3 ...  5  2  5]
 [ 1  9  4 ...  2  3  9]
 ...
 [ 3  3  1 ... 12  2  3]
 [ 4  3  2 ... 10  4  7]
 [ 3  1  3 ...  6  2 13]]
[0 1 1 ... 2 0 1]
[[ 1 11  4 ...  2  6  0]
 [ 1  1  2 ...  4  5  0]
 [ 2 13  4 ...  4  3  0]
 ...
 [ 1 10  1 ...  3 13  0]
 [ 2  6  3 ...  2 13  0]
 [ 4  6  4 ...  4 11  0]]
[[ 1 11  4 ... 13  2  6]
 [ 1  1  2 ...  2  4  5]
 [ 2 13  4 ...  4  4  3]
 ...
 [ 1 10  1 ...  9  3 13]
 [ 2  6  3 ...  2  2 13]
 [ 4  6  4 ...  8  4 11]]
[0 0 0 ... 0 0 0]


In [7]:
# print(data_train)
# print(label_train)
# tree.plot_tree(tree.DecisionTreeClassifier().fit(data_train, label_train))

In [8]:
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(data_train)
# Transform the training and testing data
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)
print(data_train)
print(data_test)

[[-1.3428396  -1.59982132 -1.34068974 ... -1.06984119 -1.34171871
   1.33939107]
 [ 0.44687963  1.33580293  0.44688334 ... -0.53539714 -0.44828522
  -0.53198733]
 [-1.3428396   0.53517814  1.34066988 ... -1.33706322  0.44514827
   0.53737175]
 ...
 [ 0.44687963 -1.06607145 -1.34068974 ...  1.33515703 -0.44828522
  -1.06666688]
 [ 1.34173925 -1.06607145 -0.4469032  ...  0.80071298  1.33858177
   0.00269221]
 [ 0.44687963 -1.59982132  0.44688334 ... -0.26817512 -0.44828522
   1.60673084]]
[[-1.3428396   1.068928    1.34066988 ...  1.60237906 -0.44828522
  -0.26464756]
 [-1.3428396  -1.59982132 -0.4469032  ... -1.33706322  1.33858177
  -0.53198733]
 [-0.44797998  1.60267787  1.34066988 ... -0.80261917  1.33858177
  -1.06666688]
 ...
 [-1.3428396   0.80205307 -1.34068974 ...  0.53349096  0.44514827
   1.60673084]
 [-0.44797998 -0.26544666  0.44688334 ... -1.33706322 -0.44828522
   1.60673084]
 [ 1.34173925 -0.26544666  1.34066988 ...  0.26626893  1.33858177
   1.0720513 ]]


In [9]:
models = [GaussianNB(), tree.DecisionTreeClassifier()]
model_names = [ "Naive Bayes", "Decision Tree"]

In [10]:
# ----------------------------------------------------------------
# Run Each Model
# ----------------------------------------------------------------
for model, name in zip(models, model_names):
    model.fit(data_train, label_train)
    # Predict
    print(data_test)
    prediction = model.predict(data_test)
    # Print Accuracy
    acc = accuracy_score(label_test, prediction)
    print("Accuracy Using", name, ": " + str(acc) + '\n')
    print(classification_report(label_test, prediction))
    print(confusion_matrix(label_test, prediction))


[[-1.3428396   1.068928    1.34066988 ...  1.60237906 -0.44828522
  -0.26464756]
 [-1.3428396  -1.59982132 -0.4469032  ... -1.33706322  1.33858177
  -0.53198733]
 [-0.44797998  1.60267787  1.34066988 ... -0.80261917  1.33858177
  -1.06666688]
 ...
 [-1.3428396   0.80205307 -1.34068974 ...  0.53349096  0.44514827
   1.60673084]
 [-0.44797998 -0.26544666  0.44688334 ... -1.33706322 -0.44828522
   1.60673084]
 [ 1.34173925 -0.26544666  1.34066988 ...  0.26626893  1.33858177
   1.0720513 ]]
Accuracy Using Naive Bayes : 0.49443333333333334

              precision    recall  f1-score   support

           0       0.49      1.00      0.66     14833
           1       0.00      0.00      0.00     12844
           2       0.00      0.00      0.00      1463
           3       0.00      0.00      0.00       630
           4       0.00      0.00      0.00       115
           5       0.00      0.00      0.00        64
           6       0.00      0.00      0.00        41
           7       0.00  

/home/juan/code/manos-poker/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
